# Download, Geocode, and Crop SAR images with a given download script

In [ ]:
import geopandas as gpd
import os
import rasterio as rio
import glob
from rasterio import plot, mask
import matplotlib.pyplot as plt
import numpy as np
import rioxarray
import gdal

In [ ]:
data_dir = 'sar_data'
raw_dir = 'sar_images_raw'
geo_dir = 'sar_images_geocoded'
geo_crop_dir = 'sar_images_geocoded_cropped'

In [ ]:
#clear current contents of folders
!rm -rf ./$data_dir/*
!rm -rf ./$raw_dir/*
!rm -rf ./$geo_dir/*
!rm -rf ./$geo_crop_dir/*

# Download

In [ ]:
#Place download script in import_sar_asf_script/ directory
download_script_name = 'download-all-2021-05-28_12-21-00.py'

In [ ]:
!cp $download_script_name ./$data_dir/

In [ ]:
!cd ./$data_dir && python $download_script_name

# Geocode

In [ ]:
for zipfile in os.listdir(data_dir):
    !unzip -j $data_dir/$zipfile '*/measurement/*vv*.tiff' -d ./$raw_dir
    !rm $data_dir/$zipfile

In [ ]:
for file in os.listdir(raw_dir):
    outfile = os.path.splitext(file)[0]+'-utm.tif'
    print(outfile)
    !gdalwarp -tps -r bilinear -tr 10 10 -srcnodata 0 -dstnodata 0 -t_srs EPSG:32605 ./$raw_dir/$file ./$geo_dir/$outfile
    !rm ./$raw_dir/$file

# Crop

In [ ]:
muldrow = gpd.read_file('../import_glacier_shapefiles_rgi/muldrow_glacier.geojson')
xmin = float(muldrow.bounds.minx)
ymin = float(muldrow.bounds.miny)
xmax = float(muldrow.bounds.maxx)
ymax = float(muldrow.bounds.maxy)

In [ ]:
for file in os.listdir(geo_dir):
    print(file)
    outfile = os.path.splitext(file)[0]+'-crop.tif'
    !gdalwarp -te $xmin $ymin $xmax $ymax $geo_dir/$file $geo_crop_dir/$outfile